In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# Install SNN dependency
!pip install snntorch

import snntorch as snn
from snntorch import surrogate

import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import KFold
import os
import copy

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
%pip install codecarbon comet_ml

In [ ]:
from comet_ml import Experiment
from codecarbon import EmissionsTracker
from datetime import datetime

# Initialise and start CodeCarbon tracker
tracker = EmissionsTracker()
tracker.start()

start_time = datetime.now()
print(f'Start time is {start_time}')

# Initialise the Comet experiment
experiment = Experiment(
    api_key="XXXXXXXXXXXXXXXXXXXXXXXXX",
    project_name="",
    workspace="",
)

In [ ]:
### Data
! wget -q https://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy

import numpy as np
MovingMNIST = np.load('mnist_test_seq.npy').transpose(1, 0, 2, 3)
MovingMNIST.shape # Custom dataset: (216, 5, 28, 28)

# Shuffle Data
np.random.shuffle(MovingMNIST)

In [ ]:
MovingMNIST_reduced = MovingMNIST[:216]
MovingMNIST_reduced.shape

In [ ]:
#Resize 64x64 images to 28x28
import torchvision.transforms as T
import torch

tf = T.Compose([
     T.ToPILImage(),
     T.Resize((28)),
     T.ToTensor()
])

resized_seqs = []
for seq in MovingMNIST_reduced:
    resized_imgs = []
    for img in seq[:5]:
        resized_img = tf(img)
        resized_imgs.append(resized_img)
    resized_imgs_stack = torch.stack(resized_imgs)
    resized_seqs.append(resized_imgs_stack)

In [ ]:
resized_seqs_stack = torch.stack(resized_seqs)
resized_seqs_stack.shape

In [ ]:
resized_seqs_reshaped = resized_seqs_stack.reshape(216, 1, 5, 28, 28)
resized_seqs_reshaped.shape

In [ ]:
### Utils
def collate(batch):
    batch = torch.stack(batch)
    batch = batch.to(device)    
    return batch[:,:,0:4], batch[:,:,4]

def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [ ]:
train_loader = DataLoader(resized_seqs_reshaped, batch_size=15, collate_fn=collate, drop_last=True)
data, target = next(iter(train_loader))
print(data.size())
print(target.size())

In [ ]:
### SConvLSTM layer
class SConvLSTMLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):

        super(SConvLSTMLayer, self).__init__()
        self.out_channels = out_channels

        # We will unroll this over time steps        
        # Params
        spike_grad_lstm = surrogate.straight_through_estimator()

        # SConvLSTM cell
        self.sclstm = snn.SConv2dLSTM(in_channels=in_channels, out_channels=out_channels, 
                        kernel_size=kernel_size, max_pool=2, spike_grad=spike_grad_lstm)

    def forward(self, X):
        # X is a frame sequence (batch_size, num_channels, seq_len, height, width)
        # Get the dimensions
        batch_size, _, seq_len, height, width = X.size()
        
        # Initialize hidden states and outputs at t=0
        syn1, mem1 = self.sclstm.init_sconv2dlstm()

        # Initialize output
        output = torch.zeros(batch_size, self.out_channels, seq_len, 
        height, width, device=device)
        
        spk_output = torch.zeros(batch_size, self.out_channels, seq_len, 
        14, 14, device=device)
        
        # Initialize Hidden State
        H = torch.zeros(batch_size, self.out_channels, 
        height, width, device=device)

        # Initialize Cell Input
        C = torch.zeros(batch_size,self.out_channels, 
        height, width, device=device)

        # Unroll over time steps
        for time_step in range(seq_len):
            spk, H, C = self.sclstm(X[:,:,time_step], H, C)
            output[:,:,time_step] = H
            spk_output[:,:,time_step] = spk
        return output, spk_output

In [ ]:
### SConvLSTM model 1 layer

class SConvLSTMModel(nn.Module):
    def __init__(self, num_channels, num_kernels, kernel_size, padding):
        super(SConvLSTMModel, self).__init__()
        
        # First csconvlstm layer
        self.sconvlstm_layer = SConvLSTMLayer(in_channels=num_channels, out_channels=num_kernels,
                                        kernel_size=kernel_size)
        # Batchnorm layer
        self.batchnorm3d = nn.BatchNorm3d(num_features=num_kernels)

        # Convolutional Layer to predict output frame
        self.conv = nn.Conv2d(
            in_channels=num_kernels, out_channels=num_channels,
            kernel_size=kernel_size, padding=padding)

    def forward(self, X):
        
        # Forward propagation through all the layers
        sconvlstm_layer_membrane, sconvlstm_layer_spikes = self.sconvlstm_layer(X)
        sconvlstm_batchnorm = self.batchnorm3d(sconvlstm_layer_membrane)
        
        # Return only the last output frame
        output = self.conv(sconvlstm_batchnorm[:,:,-1])
        spikes_output = sconvlstm_layer_spikes[:,:,-1]
        
        return nn.Sigmoid()(output), spikes_output

In [ ]:
### K-fold Cross Validator
# Params
torch.manual_seed(42)
num_epochs = 100
criterion = nn.BCELoss(reduction='sum')

# Fold results storage objects
train_start_results = {}
val_start_results = {}

train_end_results = {}
val_end_results = {}

# Per fold epoch results storage objects
train_results_per_epoch = []
val_results_per_epoch = []

train_results = []
val_results = []

spiked_targets_train = []
spiked_targets_val = []

# Define the K-fold Cross Validator
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)

# Whole dataset
dataset = resized_seqs_reshaped

# K-fold Cross Validation model evaluation
for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
    print(f'FOLD {fold}')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = SubsetRandomSampler(train_ids)
    val_subsampler = SubsetRandomSampler(val_ids)
    
    # Define data loaders for training and testing data in this fold
    train_loader = DataLoader(dataset, batch_size=15, collate_fn=collate, sampler=train_subsampler)
    val_loader = DataLoader(dataset, batch_size=15, collate_fn=collate, sampler=val_subsampler)
    
    # Initialization
    model = SConvLSTMModel(num_channels=1, num_kernels=28, kernel_size=(3, 3), padding=(1, 1)).to(device)
    model.apply(reset_weights)  
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    
    train_results_per_epoch = []
    val_results_per_epoch = []
    for epoch in range(1, num_epochs+1):
        train_loss = 0                                                 
        model.train()        
        for batch, (x, y) in enumerate(train_loader, 1):
            output, spikes = model(x)
            
            optimizer.zero_grad()
            
            loss_train = criterion(output.flatten(), y.flatten())       
            loss_train.backward()
            
            optimizer.step()
                                      
            train_loss += loss_train.item()
            #spiked_targets_train.append(spikes)
        total_train_loss = train_loss / len(train_loader.dataset)
        train_results_per_epoch.append(total_train_loss)
        
        val_loss = 0                                                 
        model.eval()                                                   
        with torch.no_grad():                                          
            for x, y in val_loader:                          
                output, spikes = model(x)                                   
                loss_val = criterion(output.flatten(), y.flatten())                
                val_loss += loss_val.item()
                #spiked_targets_val.append(spikes)
        total_val_loss = val_loss / len(val_loader.dataset)
        val_results_per_epoch.append(total_val_loss)
        
        # Store scores        
        if epoch == 1:
            val_start_results[fold] = total_val_loss
            train_start_results[fold] = total_train_loss
        else:
            val_end_results[fold] = total_val_loss
            train_end_results[fold] = total_train_loss
            
        print("Epoch:{} Training Loss:{:.2f} Validation Loss:{:.2f}\n".format(
            epoch, total_train_loss, total_val_loss))
    train_results.append(train_results_per_epoch)
    val_results.append(val_results_per_epoch)
    
    # Saving the model
    save_path = f'./convlstm-seqgmnist200-model-fold-{fold}.pth'
    torch.save(model.state_dict(), save_path)

In [ ]:
# Stop CO2 tracker and print emissions

emissions: float = tracker.stop()
print(f"Emissions: {emissions} kg")

# Calculate the time spent
stop_time = datetime.now() - start_time
time_spend = start_time - stop_time

# Time logs
experiment.log_metric("start_time", start_time) 
experiment.log_metric("stop_time", stop_time)
experiment.log_metric("time_spend", time_spend)

# Turn off Comet
experiment.end()

In [ ]:
# Print start fold results
print(f'Start K-FOLD RESULTS FOR {k_folds} FOLDS')
sum = 0.0
for key, value in train_start_results.items():
    print(f'Fold {key}: {value}')
    sum += value
print(f'Average train: {sum/len(train_start_results.items())}')

sum = 0.0
for key, value in val_start_results.items():
    print(f'Fold {key}: {value}')
    sum += value
print(f'Average val: {sum/len(val_start_results.items())}')

In [ ]:
# Print final fold results
print(f'End K-FOLD RESULTS FOR {k_folds} FOLDS')
sum = 0.0
for key, value in train_end_results.items():
    print(f'Fold {key}: {value}')
    sum += value
print(f'Average train: {sum/len(train_end_results.items())}')

sum = 0.0
for key, value in val_end_results.items():
    print(f'Fold {key}: {value}')
    sum += value
print(f'Average val: {sum/len(val_end_results.items())}')

In [ ]:
# Inference for 15 sequences
data_loader = DataLoader(dataset, batch_size=15, collate_fn=collate, drop_last=True, shuffle=True)
data, target = next(iter(data_loader))

model.eval()                                                   
with torch.no_grad():                                          
    output, spikes = model(data)

# Reshape targets and generated
targets = target.reshape(15, 28, 28, 1)
imgs_gen = output.reshape(15, 28, 28, 1)

# Join tensors for a singe image
combined = torch.cat((targets, imgs_gen), 0)
combined.shape

In [ ]:
fig = plt.figure(figsize=(15, 15))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(2, 15),  # creates 2x2 grid of axes
                 axes_pad=0.05,  # pad between axes in inch.
                 )

for ax, im in zip(grid, combined):
    # Iterating over the grid returns the Axes.
    ax.imshow(im)
    
plt.show()

In [ ]:
# Train and validation results

import matplotlib.pyplot as plt
x = list(range(0, 100))

fig, ax = plt.subplots()
t1, = ax.plot(x, train_results[0], c="blue")
t2, = ax.plot(x, train_results[1], c="brown")
t3, = ax.plot(x, train_results[2], c="green")
t4, = ax.plot(x, train_results[3], c="orange")
t5, = ax.plot(x, train_results[4], c="magenta")
v1, = ax.plot(x, val_results[0], c="blue", ls="dashed")
v2, = ax.plot(x, val_results[1], c="brown", ls="dashed")
v3, = ax.plot(x, val_results[2], c="green", ls="dashed")
v4, = ax.plot(x, val_results[3], c="orange", ls="dashed")
v5, = ax.plot(x, val_results[4], c="magenta", ls="dashed")
ax.legend((t1, t2, t3, t4, t5, v1, v2, v3, v4, v5), ('1st train fold', '2nd train fold', "3rd train fold", "4th train fold", "5th train fold", '1st val fold', '2nd val fold', "3rd val fold", "4th val fold", "5th val fold"), loc='upper right', shadow=True)
ax.set_xlabel('epochs')
ax.set_ylabel('loss')
ax.set_title('Train and validation results for 5 folds')
plt.show()

In [ ]:
# Train
fig, ax = plt.subplots()
t1, = ax.plot(x, train_results[0], c="blue")
t2, = ax.plot(x, train_results[1], c="brown")
t3, = ax.plot(x, train_results[2], c="green")
t4, = ax.plot(x, train_results[3], c="orange")
t5, = ax.plot(x, train_results[4], c="magenta")
ax.legend((t1, t2, t3, t4, t5), ('1st train fold', '2nd train fold', "3rd train fold", "4th train fold", "5th train fold"), loc='upper right', shadow=True)
ax.set_xlabel('epochs')
ax.set_ylabel('loss')
ax.set_title('Train results for 5 folds')
plt.show()

In [ ]:
# Validation
fig, ax = plt.subplots()
v1, = ax.plot(x, val_results[0], c="blue", ls="dashed")
v2, = ax.plot(x, val_results[1], c="brown", ls="dashed")
v3, = ax.plot(x, val_results[2], c="green", ls="dashed")
v4, = ax.plot(x, val_results[3], c="orange", ls="dashed")
v5, = ax.plot(x, val_results[4], c="magenta", ls="dashed")
ax.legend((v1, v2, v3, v4, v5), ("1st val fold", "2nd val fold", "3rd val fold", "4th val fold", "5th val fold"), loc='upper right', shadow=True)
ax.set_xlabel('epochs')
ax.set_ylabel('loss')
ax.set_title('Validation results for 5 folds')
plt.show()

In [ ]:
import json

with open("./" + "train_results.json", 'w') as outfile:
    json.dump(train_results, outfile)
with open("./" + "val_results.json", 'w') as outfile:
    json.dump(val_results, outfile)